# Tile_tiff Introduction, set up

https://en.wikipedia.org/wiki/Gauss%E2%80%93Boaga_projection
## Problem:
Some geotiffs are too large to be processed. For example, Azure Cognitive Services can process up to a 4mb tiff on the free tier or 50mb on the paid tier. The example tiff is 90 mb.

## Function:
This program takes a large geotiff and divides it into smaller geotiff tiles.

## Inputs:
Georeferenced map. Get the CRS and plan what CRS will be used. 

## Outputs:
A set of smaller geo referenced tifs (tiles)
tile_polys.shp - an outline of where the tiles are.

## Validation
* When done, open the tile_polys.shp and map in QGIS to make sure the area looks correct and no CRS error has been made.
* Check that the right tiles have been selected in the plot below.



In [1]:
!pip install pycodestyle pycodestyle_magic
!pip install flake8
%load_ext pycodestyle_magic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 357 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 61 kB 471 kB/s 
     |████████████████████████████████| 66 kB 4.2 MB/s 
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.13.0
    Uninstalling importlib-metadata-4.13.0:
      Successfully uninstalled importlib-metadata-4.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.4.1 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 4.2.0 which is incompatible.
gym 0.25.2 requires importlib-metadata>=4.8.0; python_version < "3.10", but you have importlib-metadata 4.2.0 which is in

In [3]:
!pip install rasterio
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Class Tiletiff

In [4]:
import sys
import os
import rasterio
import rasterio.plot
import gdal
import osr
import geopandas as gpd
from shapely.geometry import Point, Polygon
import numpy as np
import gdalnumeric
import cv2

In [62]:
class Tiletiff:
    """
    This program takes a large geotiff and divides it into smaller geotiff
    tiles. This function is useful since some geotiffs are too large for
    some uses. For example, Azure Cognitive Services can process a file
    below 4mb in size on the free tier or 50mb on the paid tier.
    The example tiff is 90 mb.

    Inputs:
    + A georeferenced tiff, such as a georeferenced map or slope image.
      Know the CRS of the tiff and plan what CRS will be used for the tiles.

    Outputs:
    + A set of smaller georeferenced tifs (tiles) stored in a directory.
    + A polygon shapefile of an outline the tiles are located.

    Validation:
    + When done, open the tile_polys.shp and original file in QGIS to make
      sure the area looks correct and no CRS error has been made.

    Attributes:
        See comments below.
    """
    # class attribute
    # the source tiff to be split into tiles
    tiff_to_tile_path = ""
    # the path to where the tiles are saved
    tt_tile_path = "/content/"

    # variables for gdal
    tt_gdal_dataset = ""
    tt_srs = ""
    tt_cols = 0
    tt_rows = 0

    # varables for rasterio
    tt_raterio_dataset = 0
    tt_crs = 0
    tt_crs_int = 0
    tt_left_min_x = 0
    tt_bottom_min_y = 0
    tt_right_max_x = 0
    tt_top_max_y = 0
    tt_coords = 0
    tt_pixel_size_x = 0
    tt_pixel_size_y = 0

    # set this for tile size
    tt_tile_pixel_width = 1000
    tt_tile_pixel_height = 1000

    # set this for overlap
    tt_tile_pixel_width_overlap = 0  # 200
    tt_tile_pixel_height_overlap = 0  # 100

    tt_tile_matrix = []
    # gdal
    tt_gdal_driver = ""
    tt_gdal_dataset = ""
    tt_gdal_dataset_band = ""
    tt_gdal_transform = ""
    tt_gdal_data = ""

    def __init__(self, tiff_to_tile_path, tt_tile_path="/content/"):
        self.tiff_to_tile_path = tiff_to_tile_path
        self.tt_tile_path = tt_tile_path

        self.tt_raterio_dataset = rasterio.open(self.tiff_to_tile_path)
        self.tt_rows, self.tt_cols = self.tt_raterio_dataset.shape
        self.tt_crs = self.tt_raterio_dataset.crs
        if (self.tt_crs.is_valid):
            self.tt_crs_int = int(str(self.tt_crs)[5:])
        else:
            self.tt_crs_int = 3857
        self.tt_srs = osr.SpatialReference()

        self.tt_srs.ImportFromEPSG(int(str(self.tt_crs_int)))

        self.tt_left_min_x = self.tt_raterio_dataset.bounds[0]
        self.tt_bottom_min_y = self.tt_raterio_dataset.bounds[1]
        self.tt_right_max_x = self.tt_raterio_dataset.bounds[2]
        self.tt_top_max_y = self.tt_raterio_dataset.bounds[3]
        self.tt_coords = [(self.tt_left_min_x, self.tt_bottom_min_y),
                          (self.tt_right_max_x, self.tt_bottom_min_y),
                          (self.tt_right_max_x, self.tt_top_max_y),
                          (self.tt_left_min_x, self.tt_top_max_y)]
        self.tt_pixel_size_x, self.tt_pixel_size_y = self.tt_raterio_dataset.res

        self.tt_gdal_driver = gdal.GetDriverByName('GTiff')
        self.tt_gdal_dataset = gdal.Open(tiff_to_tile_path)
        self.tt_gdal_dataset_band = self.tt_gdal_dataset.GetRasterBand(1)
        self.tt_gdal_transform = self.tt_gdal_dataset.GetGeoTransform()

    def create_tile_matrix(self):
        self.tt_tile_matrix = []
        number_tiles_wide = int(self.tt_cols /
                                (self.tt_tile_pixel_width
                                 - self.tt_tile_pixel_width_overlap)) + 1
        number_tiles_high = int(self.tt_rows /
                                (self.tt_tile_pixel_height
                                 - self.tt_tile_pixel_height_overlap)) + 1
        print("create_tile_matrix", number_tiles_wide, number_tiles_high)
        # rows
        for tif_rows in range(0, number_tiles_high):
            # columns
            for tif_cols in range(0, number_tiles_wide):
                # If the bottom row,
                # set tile boundary 1 whole tile up from bottom
                if tif_rows == number_tiles_high - 1:
                    lry = self.tt_rows - self.tt_tile_pixel_height
                    uly = self.tt_rows
                else:
                    lry = 0 + ((self.tt_tile_pixel_height
                                - self.tt_tile_pixel_height_overlap) * tif_rows)
                    uly = lry + self.tt_tile_pixel_height

                # If the end column,
                # set tile boundary 1 whole tile back from end
                if tif_cols == number_tiles_wide - 1:
                    lrx = self.tt_cols - self.tt_tile_pixel_width
                    ulx = self.tt_cols
                else:
                    lrx = 0 + ((self.tt_tile_pixel_width
                                - self.tt_tile_pixel_width_overlap) * tif_cols)
                    ulx = lrx + self.tt_tile_pixel_width
                # fix tile dimensions if outside of original tiff dimensions
                if (lrx > self.tt_cols):
                    lrx = self.tt_cols
                if (lry > self.tt_rows):
                    lry = self.tt_rows
                if (ulx > self.tt_cols):
                    ulx = self.tt_cols
                if (uly > self.tt_rows):
                    uly = self.tt_rows
                self.tt_tile_matrix.append([[lrx, lry],
                                            [ulx, uly],
                                            [tif_cols,
                                             tif_rows]])
        return (self.tt_tile_matrix)

    def create_tile_files(self):
        self.tt_boundary_polys = gpd.GeoDataFrame()
        self.tt_boundary_polys['geometry'] = None
        self.tt_boundary_polys.crs = ("EPSG:" + str(self.tt_crs_int))
        self.tt_boundary_polys.geometry = self.tt_boundary_polys.geometry.to_crs(crs=self.tt_crs_int)
        self.tt_boundary_polys.to_crs(crs=self.tt_crs_int)
        self.tt_boundary_polys = self.tt_boundary_polys.to_crs(epsg=self.tt_crs_int)
        for tile in self.tt_tile_matrix:
            minx = tile[0][0]
            maxx = tile[1][0]
            miny = tile[0][1]
            maxy = tile[1][1]

            tilex = "00" + str(tile[2][0])
            tilex = tilex[-2:]
            tiley = "00" + str(tile[2][1])
            tiley = tiley[-2:]

            self.tt_gdal_data = self.tt_gdal_dataset_band.ReadAsArray(minx,
                                                                      miny,
                                                                      maxx-minx,
                                                                      maxy-miny)
            output_file_name_base = "r" + tiley + "c" + tilex
            output_file_name_tiff = output_file_name_base + ".tif"
            output_file_path = os.path.join(self.tt_tile_path,
                                            output_file_name_tiff)
            print(output_file_path)
            # print(self.tt_gdal_dataset)

            self.tile_dst_ds = gdal.Translate(output_file_path,
                                              self.tt_gdal_dataset,
                                              srcWin=[minx,
                                                      miny,
                                                      maxx-minx,
                                                      maxy-miny])
            # print(self.tile_dst_ds)
            this_tile_x_min = self.tt_left_min_x + (minx * self.tt_pixel_size_x)
            this_tile_y_min = self.tt_top_max_y - (miny * self.tt_pixel_size_y)
            this_tile_x_max = self.tt_left_min_x + (maxx * self.tt_pixel_size_x)
            this_tile_y_max = self.tt_top_max_y - (maxy * self.tt_pixel_size_y)
            """
            print("this_tile_transform",
                  this_tile_x_min, self.tt_gdal_transform[1],
                  self.tt_gdal_transform[2], this_tile_y_min,
                  self.tt_gdal_transform[4],
                  self.tt_gdal_transform[5])
            """                  
            this_tile_transform = (this_tile_x_min,
                                   self.tt_gdal_transform[1],
                                   self.tt_gdal_transform[2],
                                   this_tile_y_min,
                                   self.tt_gdal_transform[4],
                                   self.tt_gdal_transform[5])
            # print("this_tile_transform2", this_tile_transform)

            # COLOR
            self.tile_dst_ds.GetRasterBand(1).SetRasterColorTable(self.tt_gdal_dataset_band.GetRasterColorTable())
            self.tile_dst_ds.GetRasterBand(1).SetRasterColorInterpretation(self.tt_gdal_dataset_band.GetRasterColorInterpretation())

            # Write metadata
            self.tile_dst_ds.SetGeoTransform(this_tile_transform)
            self.tile_dst_ds.SetProjection(self.tt_gdal_dataset.GetProjection())

            self.tile_dst_ds.GetRasterBand(1).WriteArray(self.tt_gdal_data)
            self.tile_dst_ds = None
            coords = [(this_tile_x_min, this_tile_y_min),
                      (this_tile_x_max, this_tile_y_min),
                      (this_tile_x_max, this_tile_y_max),
                      (this_tile_x_min, this_tile_y_max)]
            poly = Polygon(coords)
            new_tp_row = {'id':output_file_name_base, 'geometry':poly}
            self.tt_boundary_polys = self.tt_boundary_polys.append(new_tp_row,
                                                                   ignore_index=True)     
        self.tt_boundary_polys.to_file(os.path.join(self.tt_tile_path,
                                                    'tile_polys.shp'))
    
    def plot_tif_and_poly(self):
        # plots the source tif and the shapefile      
        poly_df = gpd.read_file(os.path.join(self.tt_tile_path,
                                             'tile_polys.shp'))
        ax = poly_df.plot(figsize=(20, 20),
                          alpha=0,
                          edgecolor='k',
                          facecolor = "white")
        rasterio.plot.show(self.tt_raterio_dataset, ax=ax)
        # plot the outlines of the tiles        
        poly_df.boundary.plot(ax=ax,color="red",alpha=0.5)
        poly_df['coords'] = poly_df['geometry'].apply(lambda x: x.representative_point().coords[:])
        poly_df['coords'] = [coords[0] for coords in poly_df['coords']]
        # display the id of each tile, example: r00c00
        for idx, row in poly_df.iterrows():
            ax.annotate(s=row['id'], xy = row['coords'],
                        horizontalalignment='center')

    def plot_tiles_and_poly(self, tiles):
        
        poly_df = gpd.read_file(os.path.join(self.tt_tile_path,
                                             'tile_polys.shp'))
        ax = poly_df.plot(figsize=(20, 20),
                          alpha=0,
                          edgecolor='k',
                          facecolor = "white")
        for tf in tiles:
            tif_file = rasterio.open(tf)
            rasterio.plot.show(tif_file, ax=ax)
        # plot the outlines of the tiles
        poly_df.boundary.plot(ax=ax,color="red",alpha=0.5)
        poly_df['coords'] = poly_df['geometry'].apply(lambda x: x.representative_point().coords[:])
        poly_df['coords'] = [coords[0] for coords in poly_df['coords']]
        # display the id of each tile, example: r00c00
        for idx, row in poly_df.iterrows():
            ax.annotate(s=row['id'], xy = row['coords'],
                        horizontalalignment='center')

    def get_attributes(self):
        return {
            "cols": str(self.tt_cols), 
            "rows": str(self.tt_rows),
            "crs": str(self.tt_crs),
            "left_min_x": str(self.tt_left_min_x),
            "bottom_min_y": str(self.tt_bottom_min_y),
            "right_max_x": str(self.tt_right_max_x),
            "top_max_y": str(self.tt_top_max_y),
            "coords": str(self.tt_coords),
            "pixel_size_x": str(self.tt_pixel_size_x),
            "pixel_size_y": str(self.tt_pixel_size_y)
            }

# Tile a map

## Steps to tile a map\

1. Set up directories. (Google Drive will be used as an example.)
2. Download the map with wget
3. Tile the map

In [13]:
import os

def check_path(fp):
    if not os.path.exists(fp):
        print("missing: ", fp)
        os.makedirs(fp)
        if not os.path.exists(fp):
            print("still missing: ", fp)
        else:
            print("made directory: ", fp)
    else:
        print("exists:", fp)

In [21]:
# fp is an abbreviation of file path
tiletiff_folder = os.path.join("/content/drive/MyDrive/","tiletiff/")
check_path(tiletiff_folder)
tiletiff_maps_folder = os.path.join(tiletiff_folder,"maps/")
check_path(tiletiff_maps_folder)
tiletiff_source_map_folder = os.path.join(tiletiff_maps_folder,"ta-map-150-iii-so/")
check_path(tiletiff_source_map_folder)
tiletiff_tiles_folder = os.path.join(tiletiff_source_map_folder,"tif_tiles/")
check_path(tiletiff_tiles_folder)

exists: /content/drive/MyDrive/tiletiff/
exists: /content/drive/MyDrive/tiletiff/maps/
exists: /content/drive/MyDrive/tiletiff/maps/ta-map-150-iii-so/
missing:  /content/drive/MyDrive/tiletiff/maps/ta-map-150-iii-so/tif_tiles/
made directory:  /content/drive/MyDrive/tiletiff/maps/ta-map-150-iii-so/tif_tiles/


In [22]:
os.chdir(tiletiff_source_map_folder)
# download a map to tile

map_file_name = "ta-map-150-iii-so-4806.tif"
map_file_path = os.path.join(tiletiff_source_map_folder, map_file_name)
if not os.path.exists(map_file_path):
    !wget https://jeffblackadar.ca/maps/ta-map-150-iii-so-4806.tif
else:
    print("File exists, so not downloading:", map_file_path)

File exists, so not downloading: /content/drive/MyDrive/tiletiff/maps/ta-map-150-iii-so/ta-map-150-iii-so-4806.tif


# Instantiate class

In [63]:
tt = Tiletiff(os.path.join(tiletiff_source_map_folder, map_file_name), 
              tiletiff_tiles_folder)

tt.tt_tile_pixel_width_overlap = 0
tt.tt_tile_pixel_height_overlap = 0
print("Preparing to tile using these attributes:")
print(tt.get_attributes())

Preparing to tile using these attributes:
{'cols': '6905', 'rows': '4904', 'crs': 'EPSG:4806', 'left_min_x': '-0.013103530018054864', 'bottom_min_y': '41.65265561049267', 'right_max_x': '0.1363657726322032', 'top_max_y': '41.75881020284559', 'coords': '[(-0.013103530018054864, 41.65265561049267), (0.1363657726322032, 41.65265561049267), (0.1363657726322032, 41.75881020284559), (-0.013103530018054864, 41.75881020284559)]', 'pixel_size_x': '2.1646531882731075e-05', 'pixel_size_y': '2.1646531882731075e-05'}


In [54]:
#tt = Tile_tiff('/content/drive/MyDrive/crane_maps_syria/maps_large/Aafrine_georef/Aafrine_georef.tif',
#               '/content/drive/MyDrive/crane_maps_syria/maps_large/Aafrine_georef/tif_tiles')
#tt = Tile_tiff('/content/drive/MyDrive/crane_maps_syria/maps_large/Djeble_georef/Djeble_georef_32637.tif',
#               '/content/drive/MyDrive/crane_maps_syria/maps_large/Djeble_georef/tif_tiles')
#tt = Tile_tiff('/content/drive/MyDrive/bsr/ta-map-149-ii-ne/ta-map-149-ii-ne_26592.tif',
#               '/content/drive/MyDrive/bsr/ta-map-149-ii-ne/tif_tiles')
#tt = Tile_tiff('/content/drive/MyDrive/bsr/ta-map-149-ii-ne/ta-map-149-ii-ne_4806.tif',
               #'/content/drive/MyDrive/bsr/ta-map-149-ii-ne/tif_tiles')

#tt = Tiletiff('/content/drive/MyDrive/bsr/ta-map-150-iii-so/ta-map-150-iii-so-4806.tif',
#               '/content/drive/MyDrive/bsr/ta-map-150-iii-so/tif_tiles')

tt = Tiletiff(os.path.join(tiletiff_source_map_folder, map_file_name), 
              tiletiff_tiles_folder)

tt.tt_tile_pixel_width_overlap = 0
tt.tt_tile_pixel_height_overlap = 0
print("Preparing to tile using these attributes:")
print(tt.get_attributes())
print(tt.create_tile_matrix())
tt.create_tile_files()

Preparing to tile using these attributes:
{'cols': '6905', 'rows': '4904', 'crs': 'EPSG:4806', 'left_min_x': '-0.013103530018054864', 'bottom_min_y': '41.65265561049267', 'right_max_x': '0.1363657726322032', 'top_max_y': '41.75881020284559', 'coords': '[(-0.013103530018054864, 41.65265561049267), (0.1363657726322032, 41.65265561049267), (0.1363657726322032, 41.75881020284559), (-0.013103530018054864, 41.75881020284559)]', 'pixel_size_x': '2.1646531882731075e-05', 'pixel_size_y': '2.1646531882731075e-05'}
create_tile_matrix 7 5
[[[0, 0], [1000, 1000], [0, 0]], [[1000, 0], [2000, 1000], [1, 0]], [[2000, 0], [3000, 1000], [2, 0]], [[3000, 0], [4000, 1000], [3, 0]], [[4000, 0], [5000, 1000], [4, 0]], [[5000, 0], [6000, 1000], [5, 0]], [[5905, 0], [6905, 1000], [6, 0]], [[0, 1000], [1000, 2000], [0, 1]], [[1000, 1000], [2000, 2000], [1, 1]], [[2000, 1000], [3000, 2000], [2, 1]], [[3000, 1000], [4000, 2000], [3, 1]], [[4000, 1000], [5000, 2000], [4, 1]], [[5000, 1000], [6000, 2000], [5, 1]],

In [64]:
tt.plot_tif_and_poly()

Output hidden; open in https://colab.research.google.com to view.


# Choose which tiles to transcribe based on row and column.

We don't need to waste time and money transcribing irrelevant tiles. 
It's possible to create an array of only the tiles we wish to process.
With this map, all of the tiles are useful. 
Other maps may have tiles with no information. 
Empty tiles can be excluded from an array of tiles.

In [65]:
tiles = []
import os
for filename in os.listdir(tiletiff_tiles_folder):
    if filename.endswith(".tif"):
        print("        tiles.append('" +
              os.path.join(tiletiff_tiles_folder, filename) +
              "')")
        tiles.append(os.path.join(tiletiff_tiles_folder, filename))

        tiles.append('/content/drive/MyDrive/tiletiff/maps/ta-map-150-iii-so/tif_tiles/r00c00.tif')
        tiles.append('/content/drive/MyDrive/tiletiff/maps/ta-map-150-iii-so/tif_tiles/r00c01.tif')
        tiles.append('/content/drive/MyDrive/tiletiff/maps/ta-map-150-iii-so/tif_tiles/r00c02.tif')
        tiles.append('/content/drive/MyDrive/tiletiff/maps/ta-map-150-iii-so/tif_tiles/r00c03.tif')
        tiles.append('/content/drive/MyDrive/tiletiff/maps/ta-map-150-iii-so/tif_tiles/r00c04.tif')
        tiles.append('/content/drive/MyDrive/tiletiff/maps/ta-map-150-iii-so/tif_tiles/r00c05.tif')
        tiles.append('/content/drive/MyDrive/tiletiff/maps/ta-map-150-iii-so/tif_tiles/r00c06.tif')
        tiles.append('/content/drive/MyDrive/tiletiff/maps/ta-map-150-iii-so/tif_tiles/r01c00.tif')
        tiles.append('/content/drive/MyDrive/tiletiff/maps/ta-map-150-iii-so/tif_tiles/r01c01.tif')
        tiles.append('/content/drive/MyDrive/tiletiff/maps/ta-map-150-iii-so/tif_tiles/r01c02.tif')


In [66]:
tt.plot_tiles_and_poly(tiles)

Output hidden; open in https://colab.research.google.com to view.